In [1]:
import pandas as pd

In [4]:
impressions_df = pd.read_csv("impressions.сsv")
impressions_df.head()

,rank,event_datetime_m,is_clicked,id_show,id_user
0,1,2017-01-12 12:30:00,0.0,5547,906335
1,3,2017-01-12 14:53:00,0.0,5547,5038628
2,4,2017-01-12 17:10:00,0.0,8148,5038628
3,2,2017-01-12 14:48:00,0.0,8131,5038628
4,1,2017-01-12 14:58:00,0.0,8367,5038628


Первый DataFrame. По аналогичному надо и делать предсказание.Что можно сказать. Придётся переделать DateTime features. Show_id и user_id в данном случае бесполезные. Если представть их как категориальные признаки, то получится супер-разреженная матрица, в которой находится около 0 полезной инфы. Довольно логично будет их выкинуть в дальнейшем обучении, а вместо них вставить инфу из других df, в которых и содержатся паттерны, уточняющие ползователей и мероприятия. Что касается признака rank, он вроде категорильаный, но с другой стороны является показателем, какое место в списке занимает билет, поэтому его можно оставить как есть(StandardScaler)

In [25]:
show_df = pd.read_csv("show_data.сsv", index_col = 'id_show')
show_df.head()

,organizer_id,age_category,duration,parent_genre_id,child_genre_id,IdBuilding,show_maxprice,show_minprice,show_meanprice,show_stdprice
id_show,,,,,,,,,,
0,1854307.0,4.0,10200.0,[13654],[13655],10.0,6000.0,160.0,1495.231712,777.042344
1,1854309.0,2.0,10800.0,[13654],[13658 13659],12.0,2600.0,150.0,995.630174,676.837725
2,1854327.0,5.0,8700.0,[13654],[13655],33.0,10000.0,50.0,743.746006,1153.720468
3,1854327.0,2.0,6600.0,[13679],[1868812],33.0,3000.0,50.0,707.560152,605.466747
4,1854319.0,7.0,6600.0,[13654],[13657],22.0,450.0,300.0,384.545227,62.711090


Вот тут мы и добрались до шоу! Оно начинается. Начнём по порядку:
<ol>
    <li>organazier_id - 1200 категорий, но с другой стороны в среднем на одного организатора приходится 25 мероприятий - не так мало. Eсть два пути - OneHot. А можно забить на этот параметр и вместо него вставить пару простых категориальных - сделать групбай, посмотреть насколько организатор популярный, дать какой-то рейтинг.</li>
    <li>age_category - в названии есть слово category... Но на самом деле он похож на чиловой. надо попробовать и так и так.</li>
    <li>duration - в половине мероприятий не указана их длительность. Наверное стоит такие классифицировать отдельно. Средняя длительность - 4000 чего-то, на самом деле средняя длительность чуть больше 8000 чего-то. Если представить, что это секунды, то 8000/60=133 минуты, чуть больше двух часов. Логистическая регрессия вряд ли будет довольна этим признаком в таком виде, как он есть. Его легко можно сделать категориальным. Ещё раз поделить по часам и взять целую часть. Супер-долгие мероприятия лучше засунуть в один класс. Надо будет посмотреть распределение на всякие bar plotах</li>
    <li>parent_genre_id and child_genre_id - OneHot так и напрашивается. Только есть один момент. Делать его в лоб нельзя. Надо посчитать сколько всего на самом деле жанров, из-за того что по дефолту двойной жанр является как бы уникальнымю Предполагаю, что их порядка 50.</li>
    <li>IdBuilding - 602 категории... Предложение сделать тоже, что и с organaizer_id</li>
    <li>price features - обычные числовые признаки, полезно посмотреть на графики. может быть как-то удачно разобьётся на классы. С последним признаком думаю стоит не париться, важное лишь отделить аномалии... 1р за билет - эито какой-то подлог:)</li>
</ol>

In [26]:
print(show_df.shape)
print(show_df['organizer_id'].nunique())
print(show_df['age_category'].value_counts())
print(show_df['duration'].describe())
print(len(show_df['duration'].nonzero()[0]))
print(show_df['parent_genre_id'].nunique())
print(show_df['child_genre_id'].nunique())
print(show_df['IdBuilding'].nunique())
print(show_df['show_maxprice'].describe())
print(show_df['show_minprice'].describe())
print(show_df['show_meanprice'].describe())
print(show_df['show_stdprice'].describe())

(30949, 10)
1198
2.0     7976
3.0     5952
1.0     5515
4.0     3611
5.0     2805
6.0      366
7.0      242
8.0      164
9.0       63
13.0      33
14.0      13
15.0      10
12.0      10
11.0       2
Name: age_category, dtype: int64
count     30025.000000
mean       3999.137386
std        6698.175723
min           0.000000
25%           0.000000
50%           0.000000
75%        7200.000000
max      216000.000000
Name: duration, dtype: float64
15499
102
901
602
count     29931.000000
mean       2552.690856
std        4652.360170
min           1.000000
25%         800.000000
50%        1500.000000
75%        3000.000000
max      200000.000000
Name: show_maxprice, dtype: float64
count    29931.00000
mean       688.37817
std        797.18843
min          0.01000
25%        300.00000
50%        500.00000
75%        800.00000
max      30000.00000
Name: show_minprice, dtype: float64
count    29931.000000
mean      1274.532005
std       1341.747581
min          1.000000
25%        529.411765
5

In [32]:
show_rate_df = pd.read_csv("show_rating.сsv")
show_rate_df.head()

,date_time,rating,rating_count,review_count,id_show
0,2013-10-15,5.0,1,NaN,10174
1,2013-11-22,5.0,1,NaN,10174
2,2013-11-24,4.0,1,NaN,10174
3,2013-11-29,4.0,1,NaN,10174
4,2013-12-13,1.0,1,1.0,10174


Довольно интересная штука. Открывая, хотел проиндексировать по show_id. Но по факту оказалось, что это вся информация об отзывах. Момент номер 1 - нам плевать на дату отзыва, когда мы покупаем билет. Именно на конкретную дату, нам важно лишь то, насколько этот отзыв релевантен. А это значит, чо надо узнать его возраст. Делается протсо - ищем нужное нам шоу в первом датафрейме и вычитаем. Цель этого датафрейма - сформировать признаки для мероприятия. Как минимум стоит посчтиать число оценок
<br>что касается самого рейтинга, то он может быть дробным. Собственно для кадого шоу у нас есть распределение оценок, а возраст оценок - это их веса. Вопрос в том, как хранить эти распределения, я пока хз
<br>rating_count... 159 оценок в сутки это ничего себе! можно для каждого шоу посчитать mean b std. Можно считать взвешенный mean(зависит от возраста)
<br>rewiew_count... Я не понял в чём отличие от предыдущего, но можно сделать тоже самое.

In [40]:
print(show_rate_df.shape)
print(show_rate_df['rating'].value_counts())
print(show_rate_df['rating_count'].value_counts())
print(show_rate_df['review_count'].value_counts())

(8635027, 5)
5.0    4791399
4.0    1602259
3.0     900295
2.0     363934
1.0     310208
4.5     265663
3.5     116297
4.3      63929
4.7      62243
2.5      32758
4.8      30836
3.7      27710
3.3      18827
3.8       9128
4.6       8509
4.2       7028
4.4       5790
1.5       3007
2.3       2835
4.9       2556
2.7       2380
3.4       1540
3.2       1522
3.6       1489
2.8       1064
4.1        565
1.8        392
1.3        248
2.1        133
3.9        120
2.4         84
1.4         76
1.7         52
1.9         41
2.6         30
2.9         24
1.2         20
2.2         19
1.6         14
1.1          2
0.0          1
Name: rating, dtype: int64
1      7115076
2      1068995
3       276528
4        94729
5        41942
6        18149
7         8960
8         3328
9         2463
10        1472
12        1127
11         774
13         417
21         370
20         166
14         165
15         132
23          87
18          75
136         71
159          1
Name: rating_count, dtype: int

In [47]:
client_df = pd.read_csv("client_data.сsv")
client_df.head(10)

,age,create_datetime,sex,id_user
0,NaN,NaN,NaN,766545
1,NaN,NaN,NaN,1319333
2,NaN,NaN,NaN,4201765
3,20.0,2014-01-28 09:41:25.556,male,543544
4,50.0,2017-01-18 22:55:37.230,female,1568031
5,NaN,NaN,NaN,3665284
6,38.0,2014-01-20 17:13:46.133,female,303489
7,NaN,NaN,NaN,376643
8,NaN,NaN,NaN,1507687
9,NaN,2014-01-20 17:26:56.823,female,375733


Что-то о юзерах мы уже знаем, а именно какие оценки они ставят. К сожалению это анонимно:( 
<br>Возраст - дааа, программисты, которые допустили отрицательные значения явно не очень. Если возраст находится в разумных пределах и отрицательный, можно взять модуль. Эта не единственная беда. Многие ставят его на отъебись. Конечно нет трёх 116 стариков, которые покупали билеты на сайте.Надо строить графики. смотреть исправлять аномалии, можно сделать категорильаные признаки, а так он обычный числовой.
<br>create_datetime - спорный момент. Думаю дата не очень важна... А вот время может оказаться показателем адекватности. Может быть стоит учитывать как давно пользователь на сайте
<br>sex - мало кто поставил, one_hot.

In [51]:
print(client_df.shape)
print(client_df['age'].value_counts())
print(client_df['sex'].value_counts())

(423358, 4)
 37.0     2308
 39.0     1013
 38.0      991
 35.0      982
 40.0      972
 33.0      972
 34.0      947
 47.0      945
 36.0      919
 31.0      884
 32.0      882
 41.0      797
 42.0      797
 30.0      782
 29.0      735
 43.0      729
 44.0      703
 45.0      678
 28.0      623
 27.0      563
 46.0      560
 48.0      490
 26.0      479
 49.0      416
 51.0      319
 50.0      308
 52.0      306
 25.0      294
 53.0      272
 24.0      266
          ... 
 3.0         5
 77.0        5
 78.0        4
 74.0        4
 79.0        3
 116.0       3
-52.0        3
 82.0        3
 4.0         2
-19.0        2
 100.0       2
 15.0        2
-50.0        2
-8.0         2
-49.0        2
 106.0       2
-45.0        1
 85.0        1
-39.0        1
 8.0         1
 9.0         1
 13.0        1
-44.0        1
-40.0        1
-38.0        1
-48.0        1
-41.0        1
 80.0        1
-1.0         1
 86.0        1
Name: age, Length: 97, dtype: int64
female    112194
male       27211
Nam

In [52]:
clicks_df = pd.read_csv("clicks_no_impressions.сsv")
clicks_df.head()

,event_datetime_m,id_show,id_user
0,2017-03-07 09:59:00,9944,1617372
1,2017-03-07 09:59:00,9944,1617372
2,2017-03-07 09:59:00,9944,1617372
3,2017-01-04 11:36:00,9944,1617372
4,2017-01-04 11:36:00,9944,1617372


Мой комп на минуту подвиc
<br>Это хоть и важное дерьмо, но наверное стоим им заняться в последнюю очередь. Явно что-то надо делать с размерами. Из этого датафрейма мы получаем 2 характеристики с возможныйми производными от них: активность пользователя и активность шоу. Для нас это возможность понять какие шоу смотрит пользователь(жанры), для шоу это понимание какая аудитория их кликает. Т.е. часть предыдущих признаков долдны быть продублирована и переосмысленна, раньше мы знали только про билеты, а теперь клики в целом. Со временем не понятно что делать...Есть идея только посчитать число кликов за сутки. и т.д.
<br>Но всё-таки для baseline решения на эту фигню можно забить. И лучше никогда не вспоминать
<br>После всего негатива можно сказать одно. если вдруг даты тестовой части пересекутся с кликами - это лик

In [53]:
print(clicks_df.shape)

(42500383, 3)


Картинки у меня не открываются. Вряд ли в них нужны нейронки, мне кажется что хватит каких-то простых фичей, как гамма, палитра и т.д. К сожалению мало знаком с извлечением фичей из картинок. надеюсь как раз познакомиться.
<br>Тестовую часть обсуждать нет смысла, там всё тоже самое. Единственное - надо взгялнуть на даты. но это позже